# Modelo Preditivo de Safras Agrícolas

Este notebook demonstra o desenvolvimento de um modelo de Machine Learning para predição de rendimento de safras agrícolas utilizando dados da Produção Agrícola Municipal (PAM) do IBGE.

## Objetivos
1. Explorar dados históricos de produção agrícola
2. Engenharia de features para agricultura
3. Treinar e comparar múltiplos modelos de ML
4. Avaliar performance e interpretar resultados

## Variável Alvo
- **Rendimento (kg/ha)**: Produtividade da cultura por hectare plantado

## 1. Configuração do Ambiente

In [ ]:
# Imports
import sys
from pathlib import Path

# Adicionar diretório raiz ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("Ambiente configurado!")

In [ ]:
# Importar módulos do projeto
from src.data import DataLoader
from src.features import FeatureEngineer
from src.models import ModelTrainer, ModelEvaluator
from src.visualization import AgricultureVisualizer
from config.settings import settings

print("Módulos do projeto carregados!")

## 2. Carregamento dos Dados

In [ ]:
# Carregar dados
loader = DataLoader()
df = loader.load_data(use_synthetic=True)

print(f"\n📊 Dataset carregado!")
print(f"   Shape: {df.shape}")
print(f"   Período: {df['ano'].min()} - {df['ano'].max()}")
print(f"   Culturas: {df['cultura'].nunique()}")
print(f"   Estados: {df['estado'].nunique()}")

In [ ]:
# Visualizar primeiros registros
df.head(10)

In [ ]:
# Informações do dataset
df.info()

In [ ]:
# Estatísticas descritivas
df.describe().round(2)

## 3. Análise Exploratória dos Dados (EDA)

In [ ]:
# Inicializar visualizador
viz = AgricultureVisualizer()

### 3.1 Distribuição da Variável Alvo

In [ ]:
# Distribuição do rendimento
fig = make_subplots(rows=1, cols=2, subplot_titles=('Distribuição', 'Box Plot por Cultura'))

fig.add_trace(
    go.Histogram(x=df['rendimento_kg_ha'], nbinsx=50, name='Rendimento'),
    row=1, col=1
)

for cultura in df['cultura'].unique():
    fig.add_trace(
        go.Box(y=df[df['cultura']==cultura]['rendimento_kg_ha'], name=cultura[:15]),
        row=1, col=2
    )

fig.update_layout(
    title='Distribuição do Rendimento (kg/ha)',
    showlegend=False,
    height=500
)
fig.show()

print(f"\nEstatísticas do Rendimento:")
print(f"  Média: {df['rendimento_kg_ha'].mean():,.0f} kg/ha")
print(f"  Mediana: {df['rendimento_kg_ha'].median():,.0f} kg/ha")
print(f"  Desvio Padrão: {df['rendimento_kg_ha'].std():,.0f} kg/ha")

### 3.2 Evolução Temporal

In [ ]:
# Evolução do rendimento por cultura
culturas_top = df['cultura'].value_counts().head(5).index.tolist()
fig = viz.plot_evolucao_culturas(df, culturas=culturas_top, metric='rendimento_kg_ha')
fig.show()

In [ ]:
# Produção total por ano
prod_anual = df.groupby('ano')['producao_ton'].sum().reset_index()

fig = px.bar(
    prod_anual,
    x='ano',
    y='producao_ton',
    title='Produção Total por Ano (toneladas)',
    labels={'ano': 'Ano', 'producao_ton': 'Produção (ton)'},
    color='producao_ton',
    color_continuous_scale='Greens'
)
fig.show()

### 3.3 Análise por Estado

In [ ]:
# Rendimento médio por estado
rend_estado = df.groupby('estado')['rendimento_kg_ha'].mean().reset_index()
rend_estado = rend_estado.sort_values('rendimento_kg_ha', ascending=True)

fig = px.bar(
    rend_estado,
    x='rendimento_kg_ha',
    y='estado',
    orientation='h',
    title='Rendimento Médio por Estado (kg/ha)',
    labels={'rendimento_kg_ha': 'Rendimento (kg/ha)', 'estado': 'Estado'},
    color='rendimento_kg_ha',
    color_continuous_scale='YlOrRd'
)
fig.update_layout(height=700)
fig.show()

### 3.4 Correlações

In [ ]:
# Matriz de correlação
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr = df[numeric_cols].corr()

fig = px.imshow(
    corr,
    title='Matriz de Correlação',
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1,
    text_auto='.2f'
)
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
# Correlações com a variável alvo
corr_target = corr['rendimento_kg_ha'].sort_values(ascending=False)
print("Correlação com Rendimento (kg/ha):")
print(corr_target)

## 4. Feature Engineering

In [ ]:
# Aplicar engenharia de features
fe = FeatureEngineer(target='rendimento_kg_ha')

# Criar features temporais
df_fe = fe.create_temporal_features(df, value_col='rendimento_kg_ha')

# Criar features de interação
df_fe = fe.create_interaction_features(df_fe)

# Criar features agregadas
df_fe = fe.create_aggregated_features(df_fe)

print(f"\nFeatures criadas!")
print(f"  Colunas originais: {len(df.columns)}")
print(f"  Colunas após FE: {len(df_fe.columns)}")
print(f"\nNovas colunas:")
new_cols = [c for c in df_fe.columns if c not in df.columns]
print(new_cols)

In [ ]:
# Visualizar features criadas
df_fe[['ano', 'estado', 'cultura', 'rendimento_kg_ha', 'rendimento_kg_ha_lag1', 
       'rendimento_kg_ha_ma3', 'rendimento_kg_ha_diff']].head(20)

## 5. Preparação dos Dados para Modelagem

In [ ]:
# Preparar dados
target = 'rendimento_kg_ha'

# Remover linhas com target nulo
df_model = df_fe.dropna(subset=[target])

# Selecionar features
feature_cols = [
    'ano', 'area_plantada_ha', 'area_colhida_ha', 'producao_ton',
    'rendimento_kg_ha_lag1', 'rendimento_kg_ha_ma3',
    'taxa_aproveitamento', 'produtividade_ton_ha'
]

# Adicionar dummies para categóricas
df_dummies = pd.get_dummies(df_model[['estado', 'cultura', 'regiao']], drop_first=True)

# Combinar features
X = pd.concat([df_model[feature_cols], df_dummies], axis=1)
y = df_model[target]

# Tratar valores nulos
X = X.fillna(X.median())

print(f"\n📊 Dados preparados:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")

In [ ]:
# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"\n📊 Split realizado:")
print(f"   Treino: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Teste: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

## 6. Treinamento de Modelos

In [ ]:
# Inicializar treinador
trainer = ModelTrainer(random_state=42)

# Modelos a treinar
models_to_train = [
    'linear_regression',
    'ridge',
    'lasso',
    'random_forest',
    'gradient_boosting'
]

# Treinar todos os modelos
print("\n🤖 Treinando modelos...\n")
trained_models = trainer.train_multiple_models(X_train, y_train, models_to_train)

In [ ]:
# Avaliar todos os modelos
print("\n📊 Avaliando modelos...\n")
results_df = trainer.evaluate_all(X_test, y_test)

print("\n" + "="*70)
print("RESULTADOS DOS MODELOS")
print("="*70)
print(results_df.to_string(index=False))

In [ ]:
# Visualizar comparação de modelos
evaluator = ModelEvaluator(output_dir='../outputs/figures')
fig = evaluator.plot_model_comparison(results_df, metric='rmse', save=False)
fig.show()

## 7. Análise do Melhor Modelo

In [ ]:
# Melhor modelo
best_model = trainer.best_model
best_model_name = trainer.best_model_name

print(f"\n🏆 Melhor Modelo: {best_model_name}")

# Predições
y_pred = best_model.predict(X_test)

In [ ]:
# Predições vs Valores Reais
fig = evaluator.plot_predictions_vs_actual(
    y_test.values, 
    y_pred,
    title=f'Predições vs Reais - {best_model_name}',
    save=False
)
fig.show()

In [ ]:
# Análise de Resíduos
fig = evaluator.plot_residuals(y_test.values, y_pred, save=False)
fig.show()

In [ ]:
# Feature Importance
importance = trainer.get_feature_importance(feature_names=X_train.columns.tolist())

if importance is not None:
    fig = evaluator.plot_feature_importance(importance, top_n=15, save=False)
    fig.show()
    
    print("\n🎯 Top 10 Features Mais Importantes:")
    print(importance.head(10).to_string(index=False))

## 8. Validação Cruzada

In [ ]:
# Cross-validation do melhor modelo
cv_results = trainer.cross_validate(
    X, y,
    model_name=best_model_name,
    cv=5,
    scoring='neg_mean_squared_error'
)

print(f"\n📊 Validação Cruzada ({best_model_name}):")
print(f"   RMSE Médio: {cv_results['mean_score']:.2f}")
print(f"   Desvio Padrão: {cv_results['std_score']:.2f}")

## 9. Salvar Modelo

In [ ]:
# Salvar melhor modelo
model_path = trainer.save_model()
print(f"\n💾 Modelo salvo em: {model_path}")

## 10. Conclusões

### Principais Descobertas:

1. **Performance dos Modelos**: Os modelos ensemble (Random Forest, Gradient Boosting) geralmente apresentam melhor performance que modelos lineares para este problema.

2. **Features Importantes**: As features temporais (lag, média móvel) são cruciais para a predição de rendimento agrícola.

3. **Padrões Identificados**:
   - O rendimento apresenta tendência de crescimento ao longo dos anos (ganhos tecnológicos)
   - Grande variação entre culturas diferentes
   - Diferenças regionais significativas

### Próximos Passos:

1. Incorporar dados climáticos (precipitação, temperatura)
2. Adicionar dados de solo
3. Testar modelos de deep learning
4. Implementar predição para safras futuras
5. Criar API para servir o modelo